In [41]:
import pandas as pd
import numpy as np
import math
import time as time
import os as os
class mazeai:
    #constructer for the class which creates class variables
    def __init__(self, sizex, sizey):
        self.x = sizex
        self.y = sizey
        #holds the map its self
        self.matrix = np.zeros((self.x,self.y))
        
        #self.costMatrix = np.zeros((self.x,self.y))
        #hold the secondary heuristic which controls when back tracking is done
        self.visitedMatrix = np.zeros((self.x,self.y))
    #add walls to the maze the walls are layed out such the knowing the distance to the node is not enough
    def addWalls(self):
        for i in range(0,3):
            self.matrix[5+i][5] = -1
        for i in range(0,3):
            self.matrix[i][5] = -1
        for i in range(0,6):
            self.matrix[i][3] = -1
    #utility functions 
    def pmaze(self):
        print()
        print(self.matrix)
    def pMatrix(self,x):
        print()
        print(x)
    def setStart(self, n, m):
        self.matrix[int(n)][int(m)] = -2
        self.aiLocationx = n
        self.aiLocationy = m
    def setEnd(self, n, m):
        self.matrix[n][m] = -3
        self.endx = n
        self.endy = m
    #will return an array containing the index of all valid potentail moves for the ai, all 
    #invalid moves will be marked by -1
    def getNiebors(self):
        x = self.x
        y = self.y
        n = self.aiLocationx
        m = self.aiLocationy
        nieborsList = np.zeros((2,self.x))
        if n+1 <= x-1:
            nieborsList[0][0] = n+1
            nieborsList[1][0] = m
        else:
            nieborsList[0][0] = -1
            nieborsList[1][0] = -1   
        if n+1 <= x-1 and m+1 <= y-1:
            nieborsList[0][1] = n+1
            nieborsList[1][1] = m+1
        else:
            nieborsList[0][1] = -1
            nieborsList[1][1] = -1
        if n+1 <= x-1 and m-1 >= 0:
            nieborsList[0][2] = n+1
            nieborsList[1][2] = m-1
        else:
            nieborsList[0][2] = -1
            nieborsList[1][2] = -1
        if m-1 >= 0:
            nieborsList[0][3] = n
            nieborsList[1][3] = m-1
        else:
            nieborsList[0][3] = -1
            nieborsList[1][3] = -1
        if m+1 <= x-1:
            nieborsList[0][4] = n
            nieborsList[1][4] = m+1
        else:
            nieborsList[0][4] = -1
            nieborsList[1][4] = -1
        if n-1 >= 0:
            nieborsList[0][5] = n-1
            nieborsList[1][5] = m
        else:
            nieborsList[0][5] = -1
            nieborsList[1][5] = -1   
        if n-1 >= 0 and m+1 <= y-1:
            nieborsList[0][6] = n-1
            nieborsList[1][6] = m+1
        else:
            nieborsList[0][6] = -1
            nieborsList[1][6] = -1
        if n-1 >= 0 and m-1 >= 0:
            nieborsList[0][7] = n-1
            nieborsList[1][7] = m-1
        else:
            nieborsList[0][7] = -1
            nieborsList[1][7] = -1
        return nieborsList
    #findes the straight line distance the ai is from the destination
    def manhattanDis(self, x):
        self.distlist = np.zeros((1,8))
        endx = self.endx
        endy = self.endy
        for i in range (0, self.x):
            
            if x[0][i] != -1:
                wallx = x[0][i]
                wally = x[1][i]
                
                wallval = self.matrix[int(wallx)][int(wally)]
                #varify that the next move is not a wall and invalid
                if(wallval != -1):
                    #get the coordinates of the candidate node
                    Valvaluex = x[0][i]
                    Valvaluey = x[1][i]
                    #get the value of the secondary huristic 
                    revisit = self.visitedMatrix[int(Valvaluex)][int(Valvaluey)]
                    #calculate the distance from the ai to its destination
                    xc = math.pow((endx-x[0][i]),2)
                    yc = math.pow((endy-x[1][i]),2)
                    #store cost calculations for each candidate node
                    self.distlist[0][i] = math.sqrt(xc + yc) + revisit
                    
        return(self.distlist)
    #takes distlist and returns the cordinates of the best move and its cost 
    def findbestmove(self, x):
        bestvaluex = self.aiLocationx
        bestvaluey = self.aiLocationy
        minmanhat = 999999999
        for i in range (0, self.x):
            if x[0][i] > -1:               
                if self.distlist[0][i] < minmanhat and self.distlist[0][i] > 0:
                    bestvaluex = x[0][i]
                    bestvaluey = x[1][i]
                    minmanhat = self.distlist[0][i]
        # updates the secondary heuristic each time a node is visited the cost the revisit it goes up by 2
        # the maximum decrease in distance is less than 2 and so nodes will only be revisited if that is 
        # the only option. This stops the algorithm from getting stopped in infinite loops
        self.visitedMatrix[int(self.aiLocationx)][int(self.aiLocationy)] += 2
        print("the revisit cost increase each time a node is visited its cost will increase by 2 for each time it is visited next")
        self.pMatrix(self.visitedMatrix)
        
        return(bestvaluex, bestvaluey, minmanhat)
        #count how many moves it takes to reach the end of the maze 
    def solveMaze(self):
        maxdepth = 99
        movecount = 0
        for i in range (0, maxdepth):
            nieborslist = self.getNiebors()
            self.manhattanDis(nieborslist)
            x,y,dist = self.findbestmove(nieborslist)
            #this line will control is you want a trail of where the ai has been printed or not
            self.matrix[int(self.aiLocationx)][int(self.aiLocationy)] = 0
            self.setStart(x,y)
            movecount = movecount+1
            print("the current maze state is")
            self.pmaze()
            print()
            print("the current possible valid states")
            self.pMatrix(nieborslist)
            print()
            print("the cost value, manhatten distence for validstates")
            self.pMatrix(self.distlist)
            time.sleep(1)
            
            if(int(dist) == 1):
                print("the maze has been solved in ", movecount," moves ")
                return
            
            
            

In [42]:
ai = mazeai(8,8)
ai.addWalls()
ai.setStart(7,7)
ai.setEnd(0,0)
ai.solveMaze()

the revisit cost increase each time a node is visited its cost will increase by 2 for each time it is visited next

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2.]]
the current maze state is

[[-3.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0. -1. -2.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.]]

the current possible valid states

[[-1. -1. -1.  7. -1.  6. -1.  6.]
 [-1. -1. -1.  6. -1.  7. -1.  6.]]

the cost value, manhatten distence for validstates

[[0.         0.         0.         9.21954446 0.         9.21954446
  0.         8.48528137]]
the revisit cost increase each time a node is visited its cost will increase by 2 for each time it is vis